In [22]:
import pandas as pd
from collections import Counter

ugene=pd.read_csv("/Applications/Unipro UGENE.app/Contents/MacOS/data/custom_annotations/plasmid_features.txt",sep="\t",header=1)
ugene['len']=ugene['Sequence'].apply(len)
ugene[ugene['Type ']=='Origin ']

,#Feature,Type,Sequence,len
249,2micron2_origin,Origin,ACGCATTTAAGCATAAACACGCACTATGCCGTTCTTCTCATGTATA...,1165
250,2micron_origin,Origin,TTATCGATGATAAGCTGTCAAAGATGAGAATTAATTCCACGGACTA...,1472
251,EBV_origin,Origin,CATTAAACGGCATGCAGGAAAAGGACAAGCAGCGAAAATTCACGCC...,2183
252,f1_origin,Origin,GCATTAAGCGCGGCGGGTGTGGTGGTTACGCGCAGCGTGACCGCTA...,307
253,oriT_RP4,Origin,gaataagggacagtgaagaaggaacacccgctcgcgggtgggccta...,99
254,p15A_origin,Origin,GCGCTAGCGGAGTGTATACTGGCTTACTATGTTGGCACTGATGAGG...,913
255,pBR322_origin,Origin,AGATCAAAGGATCTTCTTGAGATCCTTTTTTTCTGCGCGTAATCTG...,620
256,R6Kg_origin,Origin,CCATGTCAGCCGTTAAGTGTTCCTGTGTCACTCAAAATTGCTTTGA...,392
257,SV40_origin,Origin,ATCCCGCCCCTAACTCCGCCCAGTTCCGCCCATTCTCCGCCCCATG...,78


In [5]:
import subprocess
from tempfile import NamedTemporaryFile
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd

def bash(inCommand, autoOutfile = True):
    if autoOutfile == True:
        tmp = NamedTemporaryFile()
        subprocess.call(inCommand +' > '+tmp.name, shell=True)
        f = open(tmp.name,'r')
        tmp.close()
        return(f.read())
    else:
        subprocess.call(inCommand, shell=True)  
def BLAST(seq, BLASTtype="x", flags = 'bitscore sseqid pident qstart qend sseq qseq sstart send length slen qlen'): 
    query = NamedTemporaryFile()
    SeqIO.write(SeqRecord(Seq(seq), id="temp"), query.name, "fasta")
    tmp = NamedTemporaryFile()
    bash(
        'blast'+BLASTtype+' -query ' + query.name + ' -out ' + tmp.name +
        ' -db ../BLAST_dbs/addgene_CDS_20-9-3' +
        #' -db /Users/mattmcguffie/database/BLAST_dbs/fluors_20-9-2' +
        ' -culling_limit 10 ' +
        ' -max_target_seqs 1000 -outfmt "6 '+flags+'"')
    with open(tmp.name, "r") as file_handle:  #opens BLAST file
        align = file_handle.readlines()
        
    tmp.close() 
    query.close() 
    align=pd.DataFrame([ele.split() for ele in align],columns=flags.split())
    
    if not align.empty:
        #align['sseqid']=align['sseqid'].str.split("|",expand=True)[1]
        align['pident']=align['pident'].astype(float)
        align['slen']=align['slen'].astype(int)
        align['length']=align['length'].astype(int)
        align['qlen']=align['qlen'].astype(int)
        align['pmatch']=align['length']/((align['qlen']/3)-1)#because nuc->prot
        align['abspmatch']=1-abs(1-align['pmatch']) #because nuc->prot
        align['fractionmatch']=align['length']/align['slen']
        align['bitscore']=align['bitscore'].astype(float)
        align['score']=align['pident']*align['length']*align['fractionmatch']
        align=align.sort_values(by=['score'],ascending=False)
    return align 

In [17]:
csga="""atgAAACTTT TAAAAGTAGC AGCAATTGCA GCAATCGTAT TCTCCGGTAG CGCTCTGGCA
GGTGTTGTTC CTCAGTACGG CGGCGGCGGT AACCACGGTG GTGGCGGTAA TAATAGCGGC
CCAAATTCTG AGCTGAACAT TTACCAGTAC GGTGGCGGTA ACTCTGCACT TGCTCTGCAA
ACTGATGCCC GTAACTCTGA CTTGACTATT ACCCAGCATG GCGGCGGTAA TGGTGCAGAT
GTTGGTCAGG GCTCAGATGA CAGCTCAATC GATCTGACCC AACGTGGCTT CGGTAACAGC
GCTACTCTTG ATCAGTGGAA CGGCAAAAAT TCTGAAATGA CGGTTAAACA GTTCGGTGGT
GGCAACGGTG CTGCAGTTGA CCAGACTGCA TCTAACTCCT CCGTCAACGT GACTCAGGTT
GGCTTTGGTA ACAACGCGAC CGCTCATCAG TACtaa""".lower().replace(" ","").replace("\n","")

In [18]:
%time
hits=BLAST(csga)
hits

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
blastx -query /var/folders/dg/9wkj6tb16wb1x25dm2pmn2zh0000gp/T/tmp_ro2u6pi -out /var/folders/dg/9wkj6tb16wb1x25dm2pmn2zh0000gp/T/tmp_eaun41y -db ../BLAST_dbs/addgene_CDS_20-9-3 -culling_limit 10  -max_target_seqs 1000 -outfmt "6 bitscore sseqid pident qstart qend sseq qseq sstart send length slen qlen"


In [ ]:
blastx -query /Users/mattmcguffie/database/plasmids/plas_from_people/pbtk562.fasta -out ./test.blast -db ../BLAST_dbs/addgene_CDS_20-9-3 -culling_limit 10  -max_target_seqs 1000 -outfmt "6 bitscore sseqid pident qstart qend sseq qseq sstart send length slen qlen"

In [7]:
mTm=list(SeqIO.parse("/Users/mattmcguffie/database/plasmids/erm_gfp.fa", "fasta"))[0]
mTm=str(mTm.seq)

In [9]:
%time
BLAST(mTm)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


,bitscore,sseqid,pident,qstart,qend,sseq,qseq,sstart,send,length,slen,qlen,pmatch,abspmatch,fractionmatch,score
0,959.0,sp|P14752.1|,97.782,3829,5316,MNIPFVVETVLHDGLLKYKFKNSKIRSITTKPGKSKGAIFAYRSKK...,MNIPFVVETVLHDGLLKYKFKNSKIRSITTKPGKSKGAIFAYRSKS...,1,496,496,496,8961,0.166109,0.166109,1.000000,48499.872000
1,957.0,sp|P18629.1|,97.379,3829,5316,MNIPFVVETVLHDGLLKYKFKNSKIRSITTKPGKSKGAIFAYRSKK...,MNIPFVVETVLHDGLLKYKFKNSKIRSITTKPGKSKGAIFAYRSKS...,1,496,496,496,8961,0.166109,0.166109,1.000000,48299.984000
2,498.0,sp|P0A4D5.1|,100.000,1509,2243,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,1,245,245,245,8961,0.082050,0.082050,1.000000,24500.000000
3,494.0,sp|P21236.1|,99.184,1509,2243,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,1,245,245,245,8961,0.082050,0.082050,1.000000,24300.080000
4,494.0,sp|P20173.2|,99.184,1509,2243,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,1,245,245,245,8961,0.082050,0.082050,1.000000,24300.080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,57.4,sp|Q6GEH9.1|,41.975,1008,1244,KQFKTVKALILDNKVKDIIIATDAGREGELVARLILDKV---GNKK...,KQFKIVAELL--KQANTIIVATDSDREGENIAWSIIHKANAFSKDK...,82,159,81,711,8961,0.027127,0.027127,0.113924,387.338924
83,58.2,sp|Q4L8B8.1|,40.741,8902,8666,KQFKTVKSLILDKKVKEVIIATDAGREGELVARLILDKVH---NKK...,KQFKIVAELL--KQANTIIVATDSDREGENIAWSIIHKANAFSKDK...,82,159,81,711,8961,0.027127,0.027127,0.113924,375.951759
82,58.2,sp|Q4L8B8.1|,40.741,1008,1244,KQFKTVKSLILDKKVKEVIIATDAGREGELVARLILDKVH---NKK...,KQFKIVAELL--KQANTIIVATDSDREGENIAWSIIHKANAFSKDK...,82,159,81,711,8961,0.027127,0.027127,0.113924,375.951759
37,133.0,sp|P08956.3|,100.000,3,188,DLLTTGVDIPSICNIVFLRKVRSRILYEQMKGRATRLCPEVNKTSF...,DLLTTGVDIPSICNIVFLRKVRSRILYEQMKGRATRLCPEVNKTSF...,794,855,62,1170,8961,0.020764,0.020764,0.052991,328.547009


diamond blastx -d nr -q reads.fna -o matches.m8

In [65]:
import subprocess
from tempfile import NamedTemporaryFile
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd

def bash(inCommand, autoOutfile = True):
    if autoOutfile == True:
        tmp = NamedTemporaryFile()
        subprocess.call(inCommand +' > '+tmp.name, shell=True)
        f = open(tmp.name,'r')
        tmp.close()
        return(f.read())
    else:
        subprocess.call(inCommand, shell=True)  
def dBLAST(seq, db='nr_db', flags = 'bitscore sseqid pident qstart qend sseq sstart send length slen qlen'): 
    query = NamedTemporaryFile()
    SeqIO.write(SeqRecord(Seq(seq), id="temp"), query.name, "fasta")
    tmp = NamedTemporaryFile(suffix=".m8")
    bash(
        'diamond blastx -d /Users/mattmcguffie/database/BLAST_dbs/nr -q ' + 
        query.name + ' -o ' + tmp.name + ' --outfmt 6 '+flags)
    with open(tmp.name, "r") as file_handle:  #opens BLAST file
        align = file_handle.readlines()
        
    tmp.close() 
    query.close() 
    align=pd.DataFrame([ele.split() for ele in align],columns=flags.split())
    if not align.empty:
        #align['sseqid']=align['sseqid'].str.split("|",expand=True)[1]
        align['pident']=align['pident'].astype(float)
        align['slen']=align['slen'].astype(int)
        align['length']=align['length'].astype(int)
        align['qlen']=align['qlen'].astype(int)
        align['pmatch']=align['length']/align['qlen']
        align['fractionmatch']=align['length']/align['slen']
        align['bitscore']=align['bitscore'].astype(float)
        align['score']=align['pident']*align['length'].astype(int)*align['pmatch']*align['fractionmatch']
        align=align.sort_values(by=['score'],ascending=False)
    return align 

In [79]:
%time
seq='''AATGATTTTATTTTGACTGATAGTGACCTGTTCGTTGCAACAAATTGATGAGCAATGCTTTTTTATAATG
CCAACTTTGTACAAAAAAGTTGGCACCATGAGGAGCCGGAGTAACTCCGGGGTCCGGCTGGACGGCTACG
CTCGACTGGTGCAACAGACCATCCTGTGCCATCAGAATCCAGTGACTGGCTTGCTTCCAGCCAGCTATGA
TCAGAAAGATGCTTGGGTCCGAGATAATGTGTACAGCATCTTGGCTGTGTGGGGTTTGGGCCTGGCCTAT
CGGAAGAATGCAGACCGGGATGAGGATAAGGCAAAGGCCTATGAATTGGAGCAGAGTGTAGTGAAGCTGA
TGAGAGGACTACTGCACTGCATGATCAGACAGGTGGATAAAGTAGAATCCTTCAAATATAGTCAGAGTAC
TAAGGATAGCCTCCATGCAAAGTACAACACCAAAACCTGTGCCACTGTAGTGGGTGATGATCAATGGGGA
CACCTGCAGTTGGATGCTACCTCTGTGTACCTGCTCTTCTTAGCCCAAATGACTGCCTCAGGACTCCATA
TCATCCACAGCCTAGATGAAGTCAATTTCATACAGAACCTTGTGTTTTACATTGAAGCTGCATATAAAAC
TGCTGACTTCGGGATATGGGAACGTGGAGACAAGACCAACCAAGGGATCTCAGAGTTGAATGCCAGTTCA
GTTGGAATGGCAAAGGCAGCCCTGGAAGCATTAGATGAACTGGATCTGTTTGGTGTGAAAGGTGGGCCTC
AATCAGTTATCCATGTCCTGGCTGATGAAGTACAGCACTGCCAGTCTATCCTAAATTCACTACTGCCCCG
TGCTTCACATCAAAGAGTGATGCTAGTCTACTCTCAGTGGTTTCCTTCCCTGCCTTGCAGTAGAGGG'''.lower().replace(" ","").replace("\n","")
hits=dBLAST(csga)
hits

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 19.8 µs


,bitscore,sseqid,pident,qstart,qend,sseq,sstart,send,length,slen,qlen,pmatch,fractionmatch,score
0,61.6,sp|P28307|CSGA_ECOLI,100.0,1,255,MKLLKVAAIAAIVFSGSALAGVVPQXXXXXXXXXXXXXXXPNSELN...,1,85,85,151,456,0.186404,0.562914,891.897583


In [68]:
len(hits.loc[0]['sseq'])

184